# Auction algorithm for the CSP

## Problem

Let $(G, E)$ denote a directed graph. To each arc $(i, j) \in E$ we associate a cost $c_{ij}$ and a length $l_{ij}$. Let $s$ and $t$ be two distinct nodes in $G$. We consider the Constrained Shortest Path problem formulated as a mixed integer program. $A x = d$ denotes the usual network flow constraint for the shortest path:

$$
\begin{alignat*}{3}
& \text{CSP} \quad=\quad
    && \text{minimize}   \quad && c^T x \\
&   && \text{subject to} \quad && A x = d \\
&   &&                         && l^T x \leq R \\
&   &&                         && x \in \lbrace 0, 1 \rbrace^|E| \\
\end{alignat*}
$$

## Modified Dijkstra for the CSP

The modified algorithm runs Dijkstra on an auxiliary graph, which we describe here:
- Denote the auxiliary graph $(G', E')$,
- The length and cost of an arc are respectively denoted $l_{ij}$ and $c_{ij}$.
- Let $G' = \lbrace (i, l) : i \in G,\: l \leq R \text{ and there exists a path from } s \text{ to } i \text{ of length } l \rbrace$
- There is an arc in $E'$ from $(i, l_i)$ to $(j, l_j)$ in $G'$ if $l_{ij} = l_j - l_i$. This arc has cost $c_{ij}$.

In [1]:
from collections import defaultdict
from dataclasses import dataclass
import heapq
import math


@dataclass(slots=True)
class Arc:
    l: float
    c: float


Node = int
Adj = dict[Node, dict[Node, Arc]]


def sp(adj: Adj, s: Node, t: Node):
    # just dijkstra

    frontier = [(0, s, None)]
    shortest = defaultdict(lambda: math.inf)
    parent = {}

    while frontier:
        cost, node, pred = heapq.heappop(frontier)

        if node in shortest:
            continue

        if pred is not None:
            parent[node] = pred
        shortest[node] = cost

        if node == t:
            path = [t]
            while node != s:
                node = parent[node]
                path.append(node)
            return path[::-1]

        else:
            for adjacent, edge in adj[node].items():
                heapq.heappush(frontier, (cost + edge.c, adjacent, node))
    
    return None


def csp(adj: Adj, s: Node, t: Node, R: float = math.inf):
    # modified dijkstra

    frontier = [(0, 0, s, None)]
    shortest = defaultdict(lambda: math.inf)
    parent = {}

    while frontier:
        cost, distance, node, pred = heapq.heappop(frontier)

        if (node, distance) in shortest:
            continue

        if pred is not None:
            parent[node, distance] = (pred, distance - adj[pred][node].l)
        shortest[node, distance] = cost

        if node == t:
            path = [t]
            while node != s:
                node, distance = parent[node, distance]
                path.append(node)
            return path[::-1]

        else:
            for adjacent, edge in adj[node].items():
                if distance + edge.l <= R:
                    heapq.heappush(frontier, (cost + edge.c, distance + edge.l, adjacent, node))
    
    return None

## Testing the algorithm on a random grid graph

In [2]:
import random
import itertools
import collections


n = 90

nodes = range(n * n)

edges = set()
for k in nodes:
    i, j = divmod(k, n)

    for ni, nj in [(i, j + 1), (i, j - 1), (i + 1, j), (i - 1, j)]:
        if (n * ni + nj) in nodes and 0 <= nj < n:
            edges.add((k, (n * ni + nj)))
            edges.add(((n * ni + nj), k))


adj = collections.defaultdict(dict)

for (i, j) in edges:
    adj[i][j] = Arc(1, random.randint(1, 5))

In [3]:
src = dst = random.randint(0, n * n - 1)
while dst == src:
    dst = random.randint(0, n * n - 1)

path = csp(adj, src, dst, 45)

---

We find a path whose cost changes with the length constraint by brute force.

In [4]:
path = None

while path is None or path == csp(adj, src, dst):
    src = dst = random.randint(0, n * n - 1)
    while dst == src:
        dst = random.randint(0, n * n - 1)

    path = csp(adj, src, dst, 45)

In [5]:
def cost(adj, path):
    c = 0
    for i, j in zip(path, path[1:]):
        c += adj[i][j].c
    return c

spath = sp(adj, src, dst)
len(spath) - 1, len(path) - 1, cost(adj, spath), cost(adj, path)

(54, 44, 104, 111)

The constrained shortest path finds a feasible path of slightly superior cost than the unconstrained shortest path.

## A terrible benchmark

In [6]:
%%timeit

src = dst = random.randint(0, n * n - 1)
while dst == src:
    dst = random.randint(0, n * n - 1)

path = sp(adj, src, dst)

12.2 ms ± 624 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%%timeit

src = dst = random.randint(0, n * n - 1)
while dst == src:
    dst = random.randint(0, n * n - 1)

path = csp(adj, src, dst, 45)

109 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
